In [14]:
from pathlib import Path
import os
import pandas as pd
import pyperclip
import subprocess
from reload_recursive import reload_recursive
import helpers
import statsmodels.api as sm
from scipy import stats

from mri_data import file_manager as fm
from mri_data import utils

In [ ]:
drive_root = fm.get_drive_root()
dataroot = drive_root / "3Tpioneer_bids"
labelroot = drive_root / "srs-9/3Tpioneer_bids_predictions"
data_dir = Path("/home/srs-9/Projects/ms_mri/analysis/paper1/data0")

In [ ]:
df_full = pd.read_csv(data_dir / "t1_data_full.csv", index_col="subid")
pineal_label_file = dataroot / "pineal_labels.csv"
df_man = pd.read_csv(pineal_label_file, index_col="Subject")
df = df_full.loc[df_man.index, :]

for i,row in df.iterrows():
    label_path = dataroot / row['sub-ses'] / "pineal-SRS_T1.nii.gz"
    if not label_path.exists():
        label_path = dataroot / row['sub-ses'] / "pineal-SRS.nii.gz"
    
    df.loc[i, 'vol'] = utils.compute_volume(label_path)[1]
    
df = helpers.set_dz_type5(df)
df = helpers.fix_edss(df)
df = df.rename(columns={"extracted_EDSS": "EDSS"})
df = df.rename(columns={"lesion_vol_cubic": "t2lv"})
df = helpers.do_sqrt_transform(df, ["EDSS"])
df = helpers.do_sqrt_transform(df, ["t2lv"])

df = helpers.clean_df(df)
df['dzdur'] = df['dzdur'].astype("float")
numeric_vars = ["choroid_volume", "pineal_volume", "pituitary_volume", "Female", "age", "tiv", "EDSS", "EDSS_sqrt", "t2lv", "t2lv_sqrt", "dzdur", "thalamus"]
keep_cols = numeric_vars + ["dz_type5"]
df = df[keep_cols]

KeyError: 'sub-ses'

In [ ]:
df_z = df.copy()
df_z[numeric_vars] = df_z[numeric_vars].apply(stats.zscore, nan_policy="omit")

In [ ]:
model_data = df[df['dz_type5'].isin(["RMS", "PMS"])]
formula = "thalamus ~ vol + age + Female + tiv"

[282.0, 144.385727]